**Problem Set 1** — **Part 2a**: Computational Scientist

Yihan Chen

COMSCI/ECON 206 — Computational Microeconomics

Instructor: Dr. Luyao Zhang

# Abstract

In this section, I reproduce my **Public Goods Game** (PGG) from Part 1 in a computational notebook. The aim is to (i) set up the game in normal form, (ii) install the appropriate Python libraries, and (iii) verify the basic parameters of the model.

The PGG is a canonical model for studying the tension between individual incentives and collective welfare: each player has an incentive to free-ride, yet the group achieves higher welfare if all contribute. Encoding the game in Python allows us to compute equilibria algorithmically using NashPy and **QuantEcon**, which will be done in later chunks.

# 1. Installing Tools

We first upgrade `setuptools` and `pip` to ensure compatibility with external libraries. Then we install NashPy (Knight 2021) and QuantEcon (Sargent and Stachurski 2021), two standard open-source Python packages for computing Nash equilibria and analyzing strategic games. Both are required for the subsequent steps of Part 2a.

In [1]:
# install the tools you will use later
!pip install --upgrade setuptools
!pip install --upgrade pip
!pip install nashpy
!pip install quantecon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nashpy]


# 2. Defining Game Parameters

- There are three players, each with an endowment of 100.

- The multiplier is set to 1.8, giving a marginal per capita return (MPCR) of 0.6.

- Since MPCR < 1, each unit contributed reduces the contributor’s private payoff, making free-riding the individually rational strategy.

- We restrict strategies to two discrete actions: contribute 0 or 100. This binary simplification makes it straightforward to represent the normal-form game in matrices for NashPy and QuantEcon.

In [2]:
import numpy as np
import pandas as pd

# --- Game parameters from Part 1 ---
N = 3           # players
E = 100         # endowment
m = 1.8         # social multiplier
MPCR = m / N    # 0.6 < 1  -> free-riding incentive

# --- Binary actions: each player either contributes 0 or 100 ---
# We'll label actions as actual contributions, not indices
G = np.array([0, 100], dtype=int)

print(f"Players: {N}, Endowment: {E}, Multiplier m: {m}, MPCR: {MPCR} (<1 implies best response is 0 in theory)")
print("Action set for each player:", G.tolist())

Players: 3, Endowment: 100, Multiplier m: 1.8, MPCR: 0.6 (<1 implies best response is 0 in theory)
Action set for each player: [0, 100]


**Interpretation:**

This setup aligns directly with the theoretical analysis in Part 1: the unique Nash equilibrium in pure strategies is predicted to be full free-riding, i.e. contributions of zero from all players. Later cells will compute and verify this equilibrium computationally.

# 3. Building the Normal Form in QuantEcon  

In this step, I encode the 3-player Public Goods Game (PGG) into a **numeric normal-form representation** using QuantEcon.

- Each player has two possible actions: contribute **0** or contribute **100**.  
- The payoff rule is:  
$$
u_i(g_1, g_2, g_3) = (E - g_i) + \frac{m}{N}\,(g_1 + g_2 + g_3)
$$
where $E=100$, $m=1.8$, and $N=3$ (so MPCR $=0.6$).

Because QuantEcon’s `NormalFormGame` requires a payoff array where the **last axis indexes players**, I construct an array of shape `(2, 2, 2, 3)`. Each entry stores the three players’ payoffs for a given action profile. Finally, I display all 8 outcomes in a tidy DataFrame to confirm symmetry and payoff logic.

In [3]:
import quantecon.game_theory as gt

def payoff_profile(g1, g2, g3):
    total = g1 + g2 + g3
    # u_i = (E - g_i) + (m/N) * total
    u1 = (E - g1) + (m/N) * total
    u2 = (E - g2) + (m/N) * total
    u3 = (E - g3) + (m/N) * total
    return (u1, u2, u3)

# IMPORTANT: last axis must be players -> shape (2,2,2,3), dtype=float
payoff_array = np.zeros((2, 2, 2, 3), dtype=float)

for i, g1 in enumerate(G):
    for j, g2 in enumerate(G):
        for k, g3 in enumerate(G):
            payoff_array[i, j, k, :] = payoff_profile(g1, g2, g3)

# Create the NormalFormGame from the numeric payoff array
g_pgg = gt.NormalFormGame(payoff_array)

# Inspect all 8 outcomes
rows = []
for i, g1 in enumerate(G):
    for j, g2 in enumerate(G):
        for k, g3 in enumerate(G):
            u = payoff_array[i, j, k, :]
            rows.append({"g1": g1, "g2": g2, "g3": g3, "u1": u[0], "u2": u[1], "u3": u[2]})
pd.DataFrame(rows)

,g1,g2,g3,u1,u2,u3
0,0,0,0,100.0,100.0,100.0
1,0,0,100,160.0,160.0,60.0
2,0,100,0,160.0,60.0,160.0
3,0,100,100,220.0,120.0,120.0
4,100,0,0,60.0,160.0,160.0
5,100,0,100,120.0,220.0,120.0
6,100,100,0,120.0,120.0,220.0
7,100,100,100,180.0,180.0,180.0


**Interpreting**  the 8 Outcomes

- **Baseline (0,0,0):** Each player keeps the full endowment and earns 100.  
- **Full contribution (100,100,100):** Each gives up 100 but receives 0.6 × 300 = 180, so payoffs are (180,180,180).  
- **Free-riding:** If exactly one player contributes 100, that player earns 60 while the two free-riders earn 160 each.  
- **Two contributors:** If two players contribute 100, they earn 120 each while the non-contributor earns 220.  

These results illustrate the free-riding incentive: regardless of others’ actions, contributing lowers one’s private payoff because MPCR = 0.6 < 1.  
Therefore, contributing **0** is a dominant strategy, and the unique pure-strategy Nash equilibrium is **(0,0,0)**.  
This aligns with the theoretical prediction from Part 1 and sets up the equilibrium computation in the next chunk.

Next, I build a helper function `payoff_table(g3_fixed)` that generates a **normal-form payoff matrix** for the 2-player slice of the game when Player 3’s action is fixed at either 0 or 100.  

In [8]:
def payoff_table(g3_fixed):
    """Return a DataFrame showing (u1,u2,u3) triples as strings for fixed g3."""
    data = []
    for i, g1 in enumerate(G):
        row = []
        for j, g2 in enumerate(G):
            u1, u2, u3 = payoff_profile(g1, g2, g3_fixed)
            row.append(f"({int(u1)}, {int(u2)}, {int(u3)})")
        data.append(row)
    return pd.DataFrame(data,
                        index=[f"g1={a}" for a in G],
                        columns=[f"g2={a}" for a in G])

# Table when g3=0
print("Normal-form payoff matrix with g3=0")
display(payoff_table(0))

# Table when g3=100
print("Normal-form payoff matrix with g3=100")
display(payoff_table(100))

Normal-form payoff matrix with g3=0


,g2=0,g2=100
g1=0,"(100, 100, 100)","(160, 60, 160)"
g1=100,"(60, 160, 160)","(120, 120, 220)"


Normal-form payoff matrix with g3=100


,g2=0,g2=100
g1=0,"(160, 160, 60)","(220, 120, 120)"
g1=100,"(120, 220, 120)","(180, 180, 180)"


# 4. Computing Pure-Strategy Nash Equilibria (QuantEcon)

**Goal.** Use `pure_nash_brute` from QuantEcon to compute pure-strategy Nash equilibria (NE) for the 3-player PGG in its binary-action normal form.

**What this cell does.**
1. Calls `pure_nash_brute(g_pgg)` to enumerate best responses and return any pure NE as **index triples** `(i, j, k)`.
2. Maps index triples to **actual contributions** `(g1, g2, g3)` using the action grid `G = {0, 100}` and converts NumPy scalars to clean Python `int` for printing.
3. Computes **welfare** $W = u_1 + u_2 + u_3$ at the NE and at the full-contribution profile $(100,100,100)$.
4. Displays a compact **summary table** for the NE profile(s) with individual payoffs and welfare.

**Why this matters.**  
For the parameters from Part 1 ($E = 100$, $m = 1.8$, $N = 3$, so $\text{MPCR} = 0.6 < 1$), theory predicts full free-riding: $(0,0,0)$. This cell provides an **algorithmic verification** that matches the theory and prepares the required solver outputs and screenshots for Part 2a of the assignment.

In [4]:
from quantecon.game_theory import pure_nash_brute
import pandas as pd

# --- find pure NE (indices) ---
NE_idx = pure_nash_brute(g_pgg)  # list of tuples like (i, j, k)

# --- helpers for clean printing ---
to_pyints = lambda tup: tuple(int(x) for x in tup)

NE_idx_py = [to_pyints(t) for t in NE_idx]                    # indices as Python ints
NE_profiles_np = [(G[i], G[j], G[k]) for (i, j, k) in NE_idx] # contributions as numpy scalars
NE_profiles = [to_pyints(p) for p in NE_profiles_np]           # contributions as Python ints

print("Pure-strategy NE (index triples):", NE_idx_py)
print("Pure-strategy NE (contributions):", NE_profiles)

def welfare(g1, g2, g3):
    u1, u2, u3 = payoff_profile(g1, g2, g3)
    return u1 + u2 + u3

if NE_profiles:
    g1_ne, g2_ne, g3_ne = NE_profiles[0]
    print("Welfare at NE:", welfare(g1_ne, g2_ne, g3_ne))
else:
    print("No pure NE found.")

print("Welfare at full contribution (100,100,100):", welfare(100, 100, 100))

# (Optional) neat summary table
rows = []
for (g1, g2, g3) in NE_profiles:
    u1, u2, u3 = payoff_profile(g1, g2, g3)
    rows.append({"g1": g1, "g2": g2, "g3": g3, "u1": u1, "u2": u2, "u3": u3, "W": u1 + u2 + u3})
if rows:
    display(pd.DataFrame(rows))

Pure-strategy NE (index triples): [(0, 0, 0)]
Pure-strategy NE (contributions): [(0, 0, 0)]
Welfare at NE: 300.0
Welfare at full contribution (100,100,100): 540.0


,g1,g2,g3,u1,u2,u3,W
0,0,0,0,100.0,100.0,100.0,300.0


**Interpreting** the Solver’s Output

- **NE found:** `[(0, 0, 0)]` (indices and contributions match because `G = [0, 100]`).  
- **Individual payoffs at NE:** $(100, 100, 100)$ — each player keeps the endowment when nobody contributes.  
- **Welfare at NE:** $W(0,0,0) = 300$.  
- **Welfare at full contribution:** $W(100,100,100) = 540$.

**Takeaway.** With MPCR $= 0.6 < 1$, each player’s **best response is 0** regardless of others’ actions. The computed NE $(0,0,0)$ therefore **confirms the Part-1 theoretical prediction**: the equilibrium is **inefficient** relative to the social optimum (full contribution), illustrating the classic free-riding distortion. This cell’s printout and the summary table can be included as screenshots for Part 2a deliverables.


# 5. NashPy on 2-Player Slices

**What this cell checks.**  
Because NashPy is a **2-player** library, I validate the public-goods logic on 2×2 **slices** of the 3-player game by fixing Player 3’s action at $g_3 \in \{0, 100\}$ and letting Players 1–2 play $\{0, 100\}$.

In [5]:
# === Colab cell 5 (fixed & tidy) — NashPy on 2-player slices ===
import nashpy as nash
import numpy as np

# Helper: map numpy scalars to ints for nice printing
def as_int(x):
    return int(x) if isinstance(x, (np.integer,)) else x

def pretty_pure_list(pure_list):
    # pure_list like [(G[i], G[j]), ...] -> [(0, 0)] etc.
    return [(as_int(a), as_int(b)) for (a, b) in pure_list]

def pure_action_index(prob, tol=1e-9):
    """
    Return the index of a pure action if 'prob' is (numerically) one-hot,
    else return None. Works for length-2 vectors here.
    """
    prob = np.asarray(prob, dtype=float).ravel()
    # One-hot check for arbitrary length; here length=2
    idx = np.flatnonzero(prob >= 1 - tol)
    return int(idx[0]) if len(idx) == 1 else None

def two_player_slice(g3_fixed):
    # Build 2×2 payoff matrices A (P1) and B (P2) with g3 fixed
    A = np.zeros((2, 2), dtype=float)
    B = np.zeros((2, 2), dtype=float)
    for i, g1 in enumerate(G):
        for j, g2 in enumerate(G):
            u1, u2, _ = payoff_profile(g1, g2, g3_fixed)
            A[i, j] = u1
            B[i, j] = u2
    return A, B

action_labels = [as_int(a) for a in G]  # [0, 100]

for g3_fixed in [0, 100]:
    A, B = two_player_slice(g3_fixed)
    game = nash.Game(A, B)

    print(f"\n=== NashPy slice with g3 fixed at {g3_fixed} ===")
    print("Row actions (P1):", action_labels, " | Col actions (P2):", action_labels)
    print("Payoff matrix A (P1):\n", A)
    print("Payoff matrix B (P2):\n", B)

    print("Support-enumeration equilibria:")
    eqs = list(game.support_enumeration())
    if not eqs:
        print("  (none)")
    for (p, q) in eqs:
        p = np.asarray(p, dtype=float)
        q = np.asarray(q, dtype=float)
        p_list = [float(v) for v in p]
        q_list = [float(v) for v in q]
        print("  p* =", p_list, " | q* =", q_list, end="")

        # If the equilibrium is pure, translate to action labels
        i = pure_action_index(p)
        j = pure_action_index(q)
        if i is not None and j is not None:
            print(f"   -> pure actions: ({action_labels[i]}, {action_labels[j]})")
        else:
            print()

    # Brute-force pure-NE check on the slice (clean printing)
    pure_ne = []
    for i in range(2):
        for j in range(2):
            br1 = (A[:, j].argmax() == i)
            br2 = (B[i, :].argmax() == j)
            if br1 and br2:
                pure_ne.append((G[i], G[j]))
    print("Pure NE on 2p slice:", pretty_pure_list(pure_ne))


=== NashPy slice with g3 fixed at 0 ===
Row actions (P1): [0, 100]  | Col actions (P2): [0, 100]
Payoff matrix A (P1):
 [[100. 160.]
 [ 60. 120.]]
Payoff matrix B (P2):
 [[100.  60.]
 [160. 120.]]
Support-enumeration equilibria:
  p* = [1.0, 0.0]  | q* = [1.0, 0.0]   -> pure actions: (0, 0)
Pure NE on 2p slice: [(0, 0)]

=== NashPy slice with g3 fixed at 100 ===
Row actions (P1): [0, 100]  | Col actions (P2): [0, 100]
Payoff matrix A (P1):
 [[160. 220.]
 [120. 180.]]
Payoff matrix B (P2):
 [[160. 120.]
 [220. 180.]]
Support-enumeration equilibria:
  p* = [1.0, 0.0]  | q* = [1.0, 0.0]   -> pure actions: (0, 0)
Pure NE on 2p slice: [(0, 0)]


**Interpretation**

**Payoff matrices (consistency check).**  
- With $g_3 = 0$, the printed matrices are:

$$
A=\begin{bmatrix}100 & 160\\ 60 & 120\end{bmatrix},\qquad
B=\begin{bmatrix}100 & 60\\ 160 & 120\end{bmatrix}.
$$

These match the payoff rule $u_i = (E-g_i) + \frac{m}{N}(g_1+g_2+g_3)$ with $E=100,\ m=1.8,\ N=3$.

- With $g_3 = 100$, every entry increases by $\frac{m}{N}\cdot 100 = 60$ (the public good is larger), giving:

$$
A=\begin{bmatrix}160 & 220\\ 120 & 180\end{bmatrix},\qquad
B=\begin{bmatrix}160 & 120\\ 220 & 180\end{bmatrix}.
$$

This confirms the **public** (non-rival, non-excludable) nature: both players benefit when a third party contributes.

**Equilibrium results.**  
- For **both** slices ($g_3=0$ and $g_3=100$), NashPy’s support enumeration returns $p^*=[1,0]$, $q^*=[1,0]$ $\Rightarrow$ **pure actions** $(g_1,g_2)=(0,0)$.  
- The brute-force check also reports `Pure NE on 2p slice: [(0, 0)]`.

**Economic interpretation.**  
- Even when the other player contributes (or when $g_3=100$), each player’s **best response** is to **free-ride** (choose $0$) because the marginal per-capita return is $\text{MPCR} = \frac{m}{N} = 0.6 < 1$.  
- Thus, **$(0,0)$** is the unique pure-strategy NE on each 2-player slice. This is consistent with the full 3-player result **$(0,0,0)$** computed in Chunk 4.

**Takeaway.**  
The NashPy slice analysis corroborates the QuantEcon result: with $\text{MPCR} < 1$, **free-riding is individually optimal** even though total welfare would be higher if all contributed. This slice-by-slice check mirrors the instructor’s demo style and provides an additional, independent confirmation of the theory.


# Summary of Part 2a: Computational Scientist

In this Colab notebook, I implemented the **Public Goods Game (PGG)** with three players and binary actions $\{0,100\}$. Using **QuantEcon** and **NashPy**, I:

- Built the **normal-form payoff array** and verified payoffs across all 8 outcomes.  
- Computed **pure-strategy Nash equilibria** via `pure_nash_brute`, confirming the unique equilibrium at $(0,0,0)$.  
- Compared **welfare** at equilibrium ($W=300$) with the **social optimum** (full contribution, $W=540$), highlighting the efficiency loss due to free-riding.  
- Used **NashPy** on 2-player slices to double-check equilibrium logic, showing that free-riding $(0,0)$ is the best response regardless of whether the third player contributes.  

**Takeaways.**  
- With $\text{MPCR} = \tfrac{m}{N} = 0.6 < 1$, free-riding strictly dominates contribution.  
- The computational results align perfectly with the theoretical prediction from Part 1: **inefficient equilibrium vs. efficient but unstable cooperation**.  
- This exercise demonstrates the power of computational tools in confirming analytical results and producing reproducible outputs for inclusion in the assignment deliverables:contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}.  

Next steps (Part 2b) will extend the analysis by constructing the **extensive-form representation** in **Game Theory Explorer (GTE)**, solving for SPNE, and connecting sequential reasoning to simultaneous play.
